# **LABORATORIO 1 - DEEP LEARNING**

Integrantes:

a) Arturo Magno Barrantes Chuquimia 

b) Ricardo Amiel Acuña Villogas

## **Librerías**

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import Dataset, DataLoader
import torchaudio.transforms as T
from torchvision import models
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import soundfile as sf
torchaudio.set_audio_backend("soundfile")

C:\Users\Godel\AppData\Local\Temp\ipykernel_20512\3192802601.py:16: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


# **Explorando datasets de train**

In [5]:
audios = pd.read_csv('train-audios.csv')
audios.head()

,filename,SPHSUR,BOABIS,SCIPER,DENNAH,LEPLAT,RHIICT,BOALEP,BOAFAB,PHYCUV,...,SCINAS,LEPNOT,ADEMAR,BOAALM,PHYDIS,RHIORN,LEPFLA,SCIRIZ,DENELE,SCIALT
0,INCT20955_20190909_050000_0_3.wav,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,INCT20955_20190909_050000_1_4.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,INCT20955_20190909_050000_2_5.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,INCT20955_20190909_050000_3_6.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,INCT20955_20190909_050000_4_7.wav,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Extraer un sample hasta 29 porque probaremos con los primeros 30

In [6]:
audios_30 = audios.head(30)
audios_30.to_csv("train-audios-sample-first-30.csv", index=False)
audios_30

,filename,SPHSUR,BOABIS,SCIPER,DENNAH,LEPLAT,RHIICT,BOALEP,BOAFAB,PHYCUV,...,SCINAS,LEPNOT,ADEMAR,BOAALM,PHYDIS,RHIORN,LEPFLA,SCIRIZ,DENELE,SCIALT
0,INCT20955_20190909_050000_0_3.wav,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,INCT20955_20190909_050000_1_4.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,INCT20955_20190909_050000_2_5.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,INCT20955_20190909_050000_3_6.wav,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,INCT20955_20190909_050000_4_7.wav,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,INCT20955_20190909_050000_5_8.wav,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,INCT20955_20190909_050000_6_9.wav,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,INCT20955_20190909_050000_7_10.wav,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,INCT20955_20190909_050000_8_11.wav,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,INCT20955_20190909_050000_9_12.wav,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Ahora copiemos a un directorio nuevo esa muestra de 30.

In [26]:
import os
import shutil


def copy_audios(origen,destino, type="train"):
    # Asegura que el directorio de destino exista
    os.makedirs(destino, exist_ok=True)
    
    if type == "train":
        # Itera del 0 al 29 y copia cada archivo con el patrón correspondiente
        for i in range(30):
            nombre = f"INCT20955_20190909_050000_{i}_{i+3}.wav"
            ruta_origen = os.path.join(origen, nombre)
            ruta_destino = os.path.join(destino, nombre)

            if os.path.exists(ruta_origen):
                shutil.copy2(ruta_origen, ruta_destino)
                print(f"✅ Copiado: {nombre}")
            else:
                print(f"❌ No encontrado: {nombre}") 
    elif type == "test":
        # Itera del 0 al 29 y copia cada archivo con el patrón correspondiente
        for i in range(10):
            nombre = f"INCT20955_20190911_003000_{i}_{i+3}.wav"
            ruta_origen = os.path.join(origen, nombre)
            ruta_destino = os.path.join(destino, nombre)

            if os.path.exists(ruta_origen):
                shutil.copy2(ruta_origen, ruta_destino)
                print(f"✅ Copiado: {nombre}")
            else:
                print(f"❌ No encontrado: {nombre}") 
    else:
        print("Param type error")

origen_train = "audios_train/train"
destino_train = "audios_train_sample/train"

origen_test = "audios_test/test"
destino_test = "audios_test_sample/test"

copy_audios(origen_train,destino_train, type='train')
copy_audios(origen_test, destino_test, type='test')


✅ Copiado: INCT20955_20190909_050000_0_3.wav
✅ Copiado: INCT20955_20190909_050000_1_4.wav
✅ Copiado: INCT20955_20190909_050000_2_5.wav
✅ Copiado: INCT20955_20190909_050000_3_6.wav
✅ Copiado: INCT20955_20190909_050000_4_7.wav
✅ Copiado: INCT20955_20190909_050000_5_8.wav
✅ Copiado: INCT20955_20190909_050000_6_9.wav
✅ Copiado: INCT20955_20190909_050000_7_10.wav
✅ Copiado: INCT20955_20190909_050000_8_11.wav
✅ Copiado: INCT20955_20190909_050000_9_12.wav
✅ Copiado: INCT20955_20190909_050000_10_13.wav
✅ Copiado: INCT20955_20190909_050000_11_14.wav
✅ Copiado: INCT20955_20190909_050000_12_15.wav
✅ Copiado: INCT20955_20190909_050000_13_16.wav
✅ Copiado: INCT20955_20190909_050000_14_17.wav
✅ Copiado: INCT20955_20190909_050000_15_18.wav
✅ Copiado: INCT20955_20190909_050000_16_19.wav
✅ Copiado: INCT20955_20190909_050000_17_20.wav
✅ Copiado: INCT20955_20190909_050000_18_21.wav
✅ Copiado: INCT20955_20190909_050000_19_22.wav
✅ Copiado: INCT20955_20190909_050000_20_23.wav
✅ Copiado: INCT20955_20190909_

hagamos una prueba si puede leerlo

In [27]:
audio, sr = sf.read("audios_train_sample/train/INCT20955_20190909_050000_29_32.wav")
print(audio.shape, sr)  # debería ser [samples], 16000

audio, sr = sf.read("audios_test_sample/test/INCT20955_20190911_003000_9_12.wav")
print(audio.shape, sr)  # debería ser [samples], 16000

(66150,) 22050
(66150,) 22050


## **Dataset: Audios a espectrogramas**

In [12]:
class AudioDataset(Dataset):
    def __init__(self, audio_dir, csv_path=None, transform=None):
        self.audio_dir = audio_dir
        self.transform = transform
        self.mel_spec = T.MelSpectrogram(sample_rate=16000, n_mels=128)

        if csv_path:
            self.data = pd.read_csv(csv_path)
            self.files = self.data.iloc[:, 0].values
            self.labels = self.data.iloc[:, 1:].values.astype(np.float32)
            self.has_labels = True
        else:
            self.files = sorted([f for f in os.listdir(audio_dir) if f.endswith(".wav")])
            self.has_labels = False

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        filepath = os.path.join(self.audio_dir, self.files[idx])
        try:
            waveform, sr = sf.read(filepath)  # Carga con soundfile
            waveform = torch.tensor(waveform, dtype=torch.float32).unsqueeze(0)  # [1, samples]

            if sr != 16000:
                resample = T.Resample(orig_freq=sr, new_freq=16000)
                waveform = resample(waveform)

            spec = self.mel_spec(waveform)  # [1, n_mels, time]
            spec = torchaudio.functional.amplitude_to_DB(spec, multiplier=10.0, amin=1e-10, db_multiplier=0)
            spec = spec.squeeze(0).unsqueeze(0)  # [1, 128, T]

            if self.transform:
                spec = self.transform(spec)

            if self.has_labels:
                label = torch.tensor(self.labels[idx], dtype=torch.float32)
                return spec, label
            else:
                return spec, self.files[idx]

        except Exception as e:
            print(f"❌ Error al procesar {filepath}: {e}")
            return self.__getitem__((idx + 1) % len(self))  # salta al siguiente archivo válido

## **Dataset: frames/videos**

In [5]:
'''
class VideoFrameDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path, delim_whitespace=True)
        self.img_dir = img_dir
        self.transform = transform
        self.label_cols = [col for col in self.data.columns if col not in ['original_vido_id', 'video_id', 'frame_id', 'path', 'type']]
        self.labels = self.data[self.label_cols].values.astype(np.float32)
        self.paths = self.data['path'].values

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.paths[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, torch.tensor(label, dtype=torch.float32)'''

"\nclass VideoFrameDataset(Dataset):\n    def __init__(self, csv_path, img_dir, transform=None):\n        self.data = pd.read_csv(csv_path, delim_whitespace=True)\n        self.img_dir = img_dir\n        self.transform = transform\n        self.label_cols = [col for col in self.data.columns if col not in ['original_vido_id', 'video_id', 'frame_id', 'path', 'type']]\n        self.labels = self.data[self.label_cols].values.astype(np.float32)\n        self.paths = self.data['path'].values\n\n    def __len__(self):\n        return len(self.paths)\n\n    def __getitem__(self, idx):\n        img_path = os.path.join(self.img_dir, self.paths[idx])\n        image = Image.open(img_path).convert('RGB')\n        if self.transform:\n            image = self.transform(image)\n        label = self.labels[idx]\n        return image, torch.tensor(label, dtype=torch.float32)"

### **Modelo: ResNet18**

In [8]:
# Modelo CNN
class MultiLabelCNN(nn.Module):
    def __init__(self, n_classes):
        super(MultiLabelCNN, self).__init__()
        self.base = models.resnet18(pretrained=True)
        self.base.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.base.fc = nn.Sequential(
            nn.Linear(self.base.fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, n_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.base(x)

#### **Training**

In [9]:
def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluación
def eval_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Predicción para set sin etiquetas
def predict(model, dataloader, device, threshold=0.5):
    model.eval()
    results = []
    with torch.no_grad():
        for inputs, filenames in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = (outputs > threshold).float().cpu().numpy()
            for name, pred in zip(filenames, preds):
                results.append((name, pred.tolist()))
    return results

#### **Métricas (validación)** (Necesitos los test labels)

In [10]:
def evaluate_multilabel(model, dataloader, device, threshold=0.5):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = (outputs > threshold).float().cpu().numpy()
            labels = labels.cpu().numpy()
            all_preds.append(preds)
            all_labels.append(labels)

    y_pred = np.vstack(all_preds)
    y_true = np.vstack(all_labels)

    precision_macro = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)

    precision_micro = precision_score(y_true, y_pred, average='micro', zero_division=0)
    recall_micro = recall_score(y_true, y_pred, average='micro', zero_division=0)
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0)

    return {
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        "precision_micro": precision_micro,
        "recall_micro": recall_micro,
        "f1_micro": f1_micro
    }

In [29]:
n_batch = 64
n_classes = 42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiLabelCNN(n_classes=n_classes).to(device)

criterion = nn.BCEWithLogitsLoss() # Binary Cross Entropy
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Para audio o video (elige el dataset que corresponda)
# dataset = AudioDataset(...) o VideoFrameDataset(...)
#### csv=path='train-audios.csv'
#dataset = AudioDataset(csv_path='train-audios.csv',audio_dir='audios_train/train/')

#### Los primeros 30 audios del train
dataset = AudioDataset(csv_path="train-audios-sample-first-30.csv", audio_dir='audios_train_sample/train/')

#### Los primeros 30 audios del test
testset = AudioDataset(audio_dir='audios_test_sample/test/')
#dataset = VideoFrameDataset(csv_path='train-videos.csv',video_dir='videos_train/video/')

# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
dataloader = DataLoader(dataset, batch_size = n_batch, shuffle=True)
testloader = DataLoader(testset, batch_size = n_batch, shuffle=False)

# Validación
#train_audios, val_audios = train_test_split(audios, test_size=0.2, random_state=42)

#train_dataset = AudioDataset(audio_dir='audios_train/train/', df=train_audios)
#val_dataset = AudioDataset(audio_dir='audios_train/train/', df=val_audios)

#trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#valloader = DataLoader(val_dataset, batch_size=64, shuffle=False)

#Entrenamiento
for epoch in range(5):
    loss = train_model(model, dataloader, criterion, optimizer, device)
    #loss = train_model(model, train_audios, criterion, optimizer, device)
    #val = eval_model(model,val_audios, criterion, optimizer, device)
    #print(f"Época {epoch+1} - Pérdida Train: {loss:.4f} - Pérdida Validación: {val:.4f}")
    print(f"Época {epoch+1} - Pérdida Train: {loss:.4f}")
    
#val_metrics = evaluate_multilabel(model, val_audios, device)
#print(f"[Validación] Precision_macro: {val_metrics['precision_macro']:.3f}, F1_micro: {val_metrics['f1_micro']:.3f}")

c:\Users\Godel\Documents\GitHub\deep_learning\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Godel\Documents\GitHub\deep_learning\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\Godel\Documents\GitHub\deep_learning\env\Lib\site-packages\torchaudio\functional\functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


Época 1 - Pérdida Train: 0.9420
Época 2 - Pérdida Train: 0.9318
Época 3 - Pérdida Train: 0.9224
Época 4 - Pérdida Train: 0.9139
Época 5 - Pérdida Train: 0.9088


In [ ]:
#Inferencia
results = predict(model, testloader, device)
# Guardar a CSV
df_results = pd.DataFrame([
    [filename] + preds for filename, preds in results
])
columns = ['filename'] + list(dataset.data.columns[1:])  # mismo orden de etiquetas
df_results.columns = columns
df_results.to_csv("audios_predicciones_test.csv", index=False)

# Mostrar los 5 primeros resultados
for name, preds in results[:5]:
    print(f"{name} → {preds}")

INCT20955_20190911_003000_0_3.wav → [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]
INCT20955_20190911_003000_1_4.wav → [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]
INCT20955_20190911_003000_2_5.wav → [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
INCT20955_20190911_003000_3_6.wav → [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]
INCT20955_20

In [34]:
import pandas as pd

df = pd.read_csv("audios_predicciones_test.csv")
preds = df.iloc[:, 1:]  # solo las columnas de clases

# Cuántas clases predice por audio
n_labels_predichas = preds.sum(axis=1)
print("Promedio de clases activadas por audio:", n_labels_predichas.mean())

# Frecuencia por clase
print("Frecuencia de clases predichas:\n", preds.sum())

Promedio de clases activadas por audio: 14.4
Frecuencia de clases predichas:
 SPHSUR     8.0
BOABIS    10.0
SCIPER    10.0
DENNAH    10.0
LEPLAT     0.0
RHIICT     0.0
BOALEP     0.0
BOAFAB     0.0
PHYCUV     0.0
DENMIN     0.0
ELABIC     0.0
BOAPRA     0.0
DENCRU     0.0
BOALUN    10.0
BOAALB     0.0
PHYMAR     5.0
PITAZU     0.0
PHYSAU    10.0
LEPFUS    10.0
DENNAN     8.0
PHYALB     0.0
LEPLAB     1.0
SCIFUS    10.0
BOARAN     0.0
SCIFUV     0.0
AMEPIC     0.0
LEPPOD     1.0
ADEDIP     0.0
ELAMAT     0.0
PHYNAT     0.0
LEPELE     0.0
RHISCI     8.0
SCINAS     0.0
LEPNOT     1.0
ADEMAR     0.0
BOAALM     0.0
PHYDIS    10.0
RHIORN     8.0
LEPFLA     8.0
SCIRIZ     9.0
DENELE     7.0
SCIALT     0.0
dtype: float64
